# Homework 4: Predicting Solubility

In this homework, your goal is to predict the solubility of a compound, and to reach to lowest possible error on the test set. To make this homework a little more interactive, you should report your score on a scoreboard: https://keepthescore.co/board/cmdaqeorufe/ <br> For this homework, you will use a library called score, which provides you with 3 functions. 
<ul>
    <li> featurize(dataset,features): dataset should be a CSV file with at least two columns: SMILES and Solubility. features should be a list of mordred features. Returns two numpy tensors, one with the result of the feature calculation, one with the target. 
    <li> test(features,model): features should be a list of mordred features. model should be a scikit learn estimator. return the mean average error on the test set.
    <li> report(teamname): teaname should be your team name, per the canvas group assignment. Effect: records your current score on the scoreboard.
</ul>
At the end of this notebook, you will find an example on how to train a model, test it, and report a score. Note that you are allowed to use my featurize function for your featurization. If you want to use 3D features, please contact me. You should use datasets A, B, C, D, F, G, H, I for your work. You are encouraged to use multiple datasets!<br>
One final note: you will not be graded based on the scoreboard. Of course, there might be some anticorrelation between your test score and your grade, but you need not worry about the scoreboard.

In [1]:
from score import report, featurize, test

**Rules**:<br>
<ul>
    <li> It is forbidden to modify the score library;
    <li> It is forbidden to import any other function from the score library;
    <li> It is forbidden to use dataset E;
    <li> It is forbidden to use any dataset other than A, B, C, D, F, G, H, I;
    <li> It is forbidden to use features others than the ones computed from mordred;
    <li> You can use any number of features, however, at the end, you will need to provide a brief (and vague) explanation of what your features are doing.
    <li> You can use anything you want for modeling, including all the tools available in pytorch, and you can even use other machine libraries if you wish;
    <li> It is forbidden to modify the scoreboard page (be careful, you all have admin access to it).
</ul>
It is very easy to cheat and rely on your integrity to participate in good faith. If you are caught cheating, you will get 0 for the assignment. 

## Example

In [2]:
import numpy as np
import pandas as pd
import mordred.AtomCount

In [4]:
features = [mordred.AtomCount.AtomCount("X"), mordred.AtomCount.AtomCount("HeavyAtom")]

In [5]:
X_train, y_train = featurize('../../Data/Solubility/dataset-F.csv',features)

100%|██████████| 1210/1210 [00:03<00:00, 340.32it/s]


In [6]:
from sklearn.linear_model import LinearRegression

In [7]:
model = LinearRegression()

In [8]:
model.fit(X_train,y_train)

LinearRegression()

In [9]:
test(features,model)

100%|██████████| 1291/1291 [00:02<00:00, 491.51it/s]


1.1686300929090718

In [10]:
report('Test')

UnboundLocalError: local variable 'diff' referenced before assignment

## Your turn

In [3]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures as PF
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVR
import mordred
from mordred import Calculator, descriptors

In [5]:
def concatenate():
    data_name = ['B', 'C', 'D', 'F', 'G', 'H']
    df_list = []
    
    #load data
    for name in data_name: 
        df_list.append(pd.read_csv('../../Data/Solubility/dataset-'+ name +'.csv'))
        
    #concatenate all datasets
    training_data =  pd.concat(df_list, axis=0, ignore_index=True)
    training_data.to_csv('../../Data/Solubility/dataset-HW4.csv')

In [6]:
concatenate()

In [7]:
# list of mordred features
features=  [
    mordred.Weight.Weight(exact=True, averaged=False), 
    mordred.Autocorrelation.ATS(3, 'dv'), 
    mordred.Aromatic.AromaticAtomsCount, 
    mordred.ABCIndex.ABCIndex, 
    mordred.WienerIndex.WienerIndex(polarity=True), 
    mordred.TopologicalIndex.Diameter, 
    mordred.HydrogenBond.HBondAcceptor, 
    mordred.HydrogenBond.HBondDonor, 
    mordred.AtomCount.AtomCount("C"), 
    mordred.AtomCount.AtomCount("O"), 
    mordred.AtomCount.AtomCount("N"), 
    mordred.RotatableBond.RotatableBondsCount, 
    mordred.RingCount.RingCount, 
    mordred.Polarizability.APol,
    mordred.Polarizability.BPol, 
    mordred.SLogP, 
    mordred.TopoPSA.TopoPSA(True)
         ]

In [8]:
# load X_train and y_train
X_train, y_train = featurize('../../Data/Solubility/dataset-HW4.csv',features)

RDKit WARNING: [01:21:21] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [01:21:21] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [01:21:21] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [01:21:21] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [01:21:21] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [01:21:21] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [01:21:21] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [01:21:21] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [01:21:21] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [01:21:21] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [01:21:21] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [01:21:21] WARNING: not removing hydrogen atom without neighbors
  0%|          | 0/12300 [00:00<?, ?it/s

In [9]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state = 0, n_estimators=100)

In [10]:
model.fit(X_train,y_train)

RandomForestRegressor(random_state=0)

In [11]:
test(features,model)

100%|██████████| 1291/1291 [00:05<00:00, 245.83it/s]


0.12282800509067242

In [ ]:
report('Charm')

## Explanation of Used Features

1. Molecular Weight [MW]
: *Molecular weight (MW) is the mass of a compound. Generally, a heavier molecule will not be as soluble as lighter ones*

2. ATS (lag 3)
: *Autocorrelation (moreau-broto) weighted by valence electrons. Valence electrons are the most likely to be involved in bonding.*

3. AromaticAtomsCount [nAromAtom]
: *Aromatic atoms count. Increased aromaticity leads to a potential decrease in solubility.*

4. ABCIndex
: *This is an atom-bond connectivity index descriptor, and solubility relates to the connectivity of atoms.*

5. WienerIndex(polarity=true)
: *Polarity is a great indicator of solubility. We have the rule “like dissolves like” in Chemistry because polar solutes dissolve in polar solvents and nonpolar solutes dissolve in nonpolar solvents.*

6. Diameter
: *This gives the diameter of the molecule. It is more difficult for solvent molecules to surround larger solute molecules, so a larger diameter generally corresponds with lower solubility.*

7. HBondAcceptor/HBondDonor [nHBAcc/nHBDon]
: *These are the number of hydrogen bond acceptors and donors present in a molecule. This reveals information about polarity and willingness to bond.*

8. AtomCount(“C”)
: *Counts number of carbon atoms, giving information about molecular size, so would affect solubility.*

9. AtomCount(“O”)
: *Counts number of oxygen atoms, which are more polar so they would affect solubility.*

10. AtomCount(“N”)
: *Counts number of nitrogen atoms, which are more polar so they would affect solubility.* 

11. RotatableBondsCount
: *It’s giving information about bonds, how the molecule is moving in space, and this information could help predict solubility. There was a paper online that said rigidity can decrease solubility so perhaps counting rotatable bonds relates to the prediction model in this way.*

12. RingCount [nRing]
: *This is the number of rings within a molecule. Our settings include all ring types (aromatic/non-aromatic, carbon or other). Based on the type of ring and number of rings present within a molecule, the solubility will be affected. Rings can reveal information about bonding available. Aromatic rings generally decrease solubility, but substitutions onto the ring can drastically change things.*

13. APol and BPol
: *Atomic and bond polarizability. Similar to topoPSA, polarizability plays a major role in solubility.*

14. S Log(P)
: *Log S is a feature designed to predict solubility in water for a given molecule. The log portion refers to the log of the molarity of a molecule. Filter It is a program that filters out molecules with undesirable qualities.*

15. TopoPSA:
: *Topological surface area is the sum of all polar regions of a molecule (oxygens, nitrogens, and attached hydrogens). This is basically a map of polarity (ie “like dissolves like” and to what degree this would happen in a particular situation).*